In [ ]:
!nvidia-smi

Sat Nov  9 19:10:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install numpy pandas tensorflow scikit-learn tqdm
!pip install tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path='/content/drive/MyDrive/cs challenge/cs challenge/NSL-KDD-Dataset-master/'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random
from collections import deque
import time
from tqdm import tqdm
import subprocess

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.73    # discount factor (from paper)
        self.epsilon = 0.5   # exploration rate (from paper)
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.5  # from paper
        self.model = self._build_model()
        self.attack_therapy = {
            0: "DoS",   # Denial of Service
            1: "Probe",  # Probe
            2: "U2R",    # User to Root
            3: "R2L"     # Remote to Local
        }

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return  # Do not perform replay if there aren't enough experiences in memory

        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.max(self.model.predict(next_state))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)

    def apply_therapy(self, action):
        # Translate action to attack therapy
        attack_type = self.attack_therapy.get(action, "Unknown")

        # Apply firewall therapy based on attack type
        if attack_type == "DoS":
            self.dos_therapy()
        elif attack_type == "Probe":
            self.probe_therapy()
        elif attack_type == "U2R":
            self.u2r_therapy()
        elif attack_type == "R2L":
            self.r2l_therapy()
      #### we work with coll So we just print the actions that our model do ###
    def dos_therapy(self):
        """Increase firewall rate limiting and block IP"""
        print("Applying DoS Therapy: Increasing rate limiting and blocking IP")
        print("command = New-NetFirewallRule -DisplayName 'DoS Block' -Direction Inbound -Action Block -Protocol TCP -LocalPort 80")
        print("self._execute_powershell_command(command)")

    def probe_therapy(self):
        """Enable stricter logging and detection"""
        print("Applying Probe Therapy: Enabling stricter logging and detection")
        print("command = Set-NetFirewallProfile -All -LogFileName 'C:\\FirewallLogs\\firewall.log' -LogMaxSize 4096")
        print("self._execute_powershell_command(command)")

    def u2r_therapy(self):
        """Isolate affected system and reset root password"""
        print("Applying U2R Therapy: Isolating affected system and resetting root password")
        # Simulating system isolation by blocking IP and resetting root password (not realistic in all systems)
        print("command_block = New-NetFirewallRule -DisplayName 'U2R Block' -Direction Inbound -Action Block -RemoteAddress 'affected_ip'")
        print("command_reset = net user Administrator /active:no")  # Deactivating Administrator account
        print("self._execute_powershell_command(command_block)")
        print("self._execute_powershell_command(command_reset)")

    def r2l_therapy(self):
        """Strengthen login security and monitor unauthorized access"""
        print("Applying R2L Therapy: Strengthening login security and monitoring unauthorized access")
        print("command_2fa = Set-LocalUser -Name 'Administrator' -PasswordNeverExpires $true")
        print("command_monitoring = Set-NetFirewallProfile -All -AllowInboundRemoteDesktop $false")  # Blocking remote access
        print("self._execute_powershell_command(command_2fa)")
        print("self._execute_powershell_command(command_monitoring)")

    def _execute_powershell_command(self, command):
        """Execute PowerShell command from Python"""
        print("process = subprocess.Popen([powershell, -Command, command], stdout=subprocess.PIPE, stderr=subprocess.PIPE)")
        print("out, err = process.communicate()")
        #if err:
           # print(f"Error: {err.decode()}")
       # else:
           # print(f"Output: {out.decode()}")

def preprocess_data(file_path):
    # Read the NSL-KDD dataset
    columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
               'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
               'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
               'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
               'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate',
               'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate',
               'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
               'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
               'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
               'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
               'dst_host_srv_rerror_rate', 'class', 'extra_feature']

    df = pd.read_csv(file_path, names=columns)

    # Convert categorical variables to numeric
    categorical_columns = ['protocol_type', 'service', 'flag']
    df = pd.get_dummies(df, columns=categorical_columns)

    # Separate features and labels
    X = df.drop(['class', 'extra_feature'], axis=1)
    y = df['class']

    # Convert labels to binary (normal=0, attack=1)
    y = (y != 'normal').astype(int)

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    return X_scaled, y

In [1]:
import time  # For tracking time
from tqdm import tqdm  # For a progress bar (optional)

def train_dqn(X_train, y_train, episodes=1):
    state_size = X_train.shape[1]
    action_size = 2  # fix or escalate
    agent = DQNAgent(state_size, action_size)
    batch_size = 8 #############batch size  a changer 8 / 16

    results = []

    for episode in range(episodes):
        total_reward = 0
        correct_predictions = 0

        # Add a progress bar here  800 a changer vers toute la data
        with tqdm(total=800, desc=f"Episode {episode+1}/{episodes}", unit="sample") as pbar:
            for i in range(800):  # Train on only 200 samples
                start_time = time.time()  # Start time for each sample

                state = X_train[i].reshape(1, -1)
                action = agent.act(state)
                reward = 1 if action == y_train[i] else 0

                # For the last state, we'll use the same state as next_state
                next_state = X_train[i+1].reshape(1, -1) if i < 199 else state
                done = i == 199

                agent.remember(state, action, reward, next_state, done)
                total_reward += reward

                if action == y_train[i]:
                    correct_predictions += 1

                if len(agent.memory) > batch_size:
                    agent.replay(batch_size)

                # Measure time elapsed for the current sample
                sample_time = time.time() - start_time

                # Print details for each sample (optional, can be removed for brevity)
                print(f"Episode: {episode}, Sample: {i}/{800}, Accuracy: {correct_predictions/(i+1):.4f}, Reward: {reward}, Time per Sample: {sample_time:.4f} sec")

                pbar.update(1)  # Update progress bar

        accuracy = correct_predictions / 800
        results.append({
            'episode': episode,
            'accuracy': accuracy,
            'total_reward': total_reward
        })
        print(f"Episode: {episode}, Accuracy: {accuracy:.4f}, Total Reward: {total_reward}")

    # Save the trained model
    agent.model.save('dqn_model_800_samples.keras')

    return agent, results


# Main execution
if __name__ == "__main__":
    # Load and preprocess training data
    X_train, y_train = preprocess_data(path+'KDDTrain+.txt')

    # Train the model on only 200 samples
    agent, results = train_dqn(X_train[:800], y_train[:800], episodes=7)  # Train on only the first 200 samples

    # Convert results to DataFrame for easy analysis
    results_df = pd.DataFrame(results)
    print("\nTraining Results:")
    print(results_df)
